In [34]:
import pandas as pd, numpy as np, re, string
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer as tf
train = pd.read_csv("C:\\SpringBoard\\CapstoneOne\\RedditTrain.csv", encoding='utf-8')
test = pd.read_csv("C:\\SpringBoard\\CapstoneOne\\RedditTest.csv", encoding='utf-8')

In [35]:
train.head()

,Unnamed: 0,comment,positive,negative,compund,positive_comment,negative_comment,neutral_comment
0,0,"""You must post a clear and direct question in ...",0.090,0.052,0.1531,1,0,0
1,1,I have learned English.,0.000,0.000,0.0000,0,0,1
2,2,How to bunny hop,0.000,0.000,0.0000,0,0,1
3,3,"""You laugh, but have you ever realized you wal...",0.099,0.000,0.3182,1,0,0
4,4,After getting shot I can heal myself by drinki...,0.189,0.000,0.2732,1,0,0


In [36]:
test.head()

,Unnamed: 0,comment,positive,negative,compund,positive_comment,negative_comment,neutral_comment
0,5298,Yep. Wood=less deadly confirmed,0.423,0.000,0.2960,1,0,0
1,5299,They are discriminating against them based on ...,0.038,0.160,-0.7713,0,1,0
2,5300,Did you just sit here with a straight face and...,0.083,0.131,-0.2748,0,1,0
3,5301,It is less damaging unless you’re talking abou...,0.000,0.188,-0.4601,0,1,0
4,5302,"""I'm not saying I want another mass shooting, ...",0.374,0.029,0.9359,1,0,0


In [37]:
stats = train.comment.str.len()
stats.mean(), stats.std(), stats.max()

(130.1136851136851, 124.85501546472132, 1493.0)

In [38]:
#Get rid of empty comments
train['comment'].fillna("unknown",inplace=True)
test['comment'].fillna("unknown", inplace=True)
#label columns
label_cols = ['positive_comment','negative_comment','neutral_comment']

In [39]:
#Build Model creating bag of words representation
re_token = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_token.sub(r'\1',s).split()
n = train.shape[0]
tf
vect = tf(ngram_range=(1,2), tokenizer=tokenize, min_df=3,
                      max_df=.9, strip_accents='unicode',use_idf=1,
                      smooth_idf=1, sublinear_tf=1)
train_term_doc = vect.fit_transform(train['comment'])
test_term_doc = vect.transform(test['comment'])

In [40]:
train_term_doc

<5298x8890 sparse matrix of type '<class 'numpy.float64'>'
	with 116561 stored elements in Compressed Sparse Row format>

In [41]:
test_term_doc

<2513x8890 sparse matrix of type '<class 'numpy.float64'>'
	with 46033 stored elements in Compressed Sparse Row format>

In [42]:
#basic Naive Bayes feature equation
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1)/((y==y_i).sum()+1)
#fit model for one category comment at a time
def model(y):
    y = y.values
    r = np.log(pr(1,y)/pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [43]:
x = train_term_doc
test_x = test_term_doc

In [44]:
print(test_x)

  (0, 2126)	0.7316648331893637
  (0, 1939)	0.6816645596435101
  (1, 8690)	0.10032635768575522
  (1, 8479)	0.10558256944816731
  (1, 8173)	0.13243970433895522
  (1, 7732)	0.14455990351164674
  (1, 7730)	0.05488239296086236
  (1, 7546)	0.181123519811459
  (1, 7497)	0.14012004713382398
  (1, 7491)	0.1959788226459939
  (1, 7409)	0.11065335356582902
  (1, 7396)	0.10040865127500188
  (1, 6954)	0.06694642037279754
  (1, 6921)	0.21598492784323411
  (1, 6474)	0.2012250200492946
  (1, 6469)	0.15334661292698523
  (1, 6190)	0.20529075563016064
  (1, 6053)	0.176314775267181
  (1, 5453)	0.172249039686315
  (1, 5445)	0.19181769202453247
  (1, 5429)	0.10518444756520574
  (1, 5291)	0.17782233265943748
  (1, 5264)	0.08436189929275098
  (1, 4409)	0.20529075563016064
  (1, 4408)	0.181123519811459
  :	:
  (2511, 6783)	0.20273328009690622
  (2511, 5601)	0.248347495590195
  (2511, 5599)	0.163329872963798
  (2511, 5577)	0.20109358131050625
  (2511, 5538)	0.17563197233957797
  (2511, 5027)	0.12808368413186752


In [45]:
#Don't run till we have test set
preds = np.zeros((len(test), len(label_cols)))
for i, j in enumerate(label_cols):
    print('fit',j)
    m,r = model(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit positive_comment
fit negative_comment
fit neutral_comment


In [76]:
preds_df = pd.DataFrame(preds)
preds_df.to_csv("test_results.csv", encoding='utf-8', index=False)

In [77]:
#submid = pd.DataFrame({'id': subm["id"]})
#submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
#submission.to_csv('submission.csv', index=False)



In [88]:
#from sklearn.metrics import accuracy_score
#test_results = pd.read_csv("C:\\SpringBoard\\CapstoneOne\\RedditTestAccuracy.csv", encoding='utf-8')
#label_probs = ['positive_prob','negative_prob','neutral_prob']
#accuracy_score(test_results[label_cols],test_results[label_probs])

In [89]:
 #from sklearn.naive_bayes import GaussianNB
   # clf = GaussianNB()
    #clf.fit(train,label_cols)
   # pred = clf.predict(features_test)